Author: Rudi Kreidenhuber, <Rudi.Kreidenhuber@gmail.com>, 
License: BSD (3-clause)

----

# Video EEG Monitoring Annotation visualizer

----


## Inputs:
 - .edf-files you wish to analyze go into ./data folder

## Run:
 - Press play :-)

## Outputs:
 - Found in results folder
 - Results for single files are put into a folder that matches the input-filename

----

## Howto:
 1. **Mark Events in EEG file using the following prefixes:**
 - e- --> EEG marker
 - s- --> Semiology marker
 - no prefix --> Everything else (clinical tests during/ after seizure)
 - i- --> Marker to ignore for focused analysis

 - One marker **must (!) contain "Beginn"** --> this is considered the seizure onset (if it is missing, onset is set to zero)
 - every marker **can** contain Beginn, for example:
 - Onset first seen in EEG --> Markername "e-asdBeginnfgh" --> would still be recognized as EEG marker and seizure onset
 2. **Save EEG file in .edf format and copy to ./data folder**
 - Every file in this folder is going to be analyzed, if it ends with .edf
 

In [2]:
# general imports
import os
from os.path import join
import glob
import mne
import re
from mne import Report
import pandas as pd
import numpy as np
from utils import (get_parent_dir, extract_lab_sec, raw_to_df, extract_ordered_groups, save_plotly_to_html,
                        create_results_folders, plot_interactive_subplot_with_table,
                        plot_interactive_tables, plot_interactive_eeg_and_semio, plot_interactive_eventcount,
                        plot_interactive_testing_results, plot_interactive_EEG_results, plot_interactive_semio_results,
                        win_create_results_folders, write_excel_table)

# plotly imports
import plotly as py
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
py.offline.init_notebook_mode(connected=True)

win = False

# grab .edfs
edfs = glob.glob("../data/*.edf")
if win:
    edfs = glob.glob("..\\data\\*.edf")

print("edfs found:\n", edfs)

if win:
    win_create_results_folders(edfs)
    subj_name =  os.getcwd().split("\\")[-2].split("VEEG_Event_Processor-")[-1]
else:
    create_results_folders(edfs)
    subj_name =  os.getcwd().split("/")[-2].split("VEEG_Event_Processor-")[-1]
print(f"Subject/ Patient name is: {subj_name}")

#    find configuration file
config_file = os.path.join(os.path.getcwd(), "VEEG_config.xlsx")
if os.path.isfile(config_file):
    print(f"Using configuration file: {config_file}")

ModuleNotFoundError: No module named 'pandas'

----
## Save data
----

In [ ]:
df = dict() 
e_events = dict()
s_events = dict()
t_events = dict()


for e in edfs:
    print(f"Now processing file: {e}")
    raw = mne.io.read_raw(e, preload=True)
    df[e], onset = raw_to_df(raw, e)

    e_events[e], s_events[e], t_events[e] = extract_ordered_groups(df[e]) 
    
    #save
    if win:
        csv_path = os.path.join("..", "results", e.split("\\")[-1].split(".")[0], "tables")
        e_file = e.split("\\")[-1].split(".")[0]
    else:
        csv_path = os.path.join("..", "results", e.split("/")[-1].split(".")[0], "tables")
        e_file = e.split("/")[-1].split(".")[0]
    tsv_name = "All_data_" + e_file + ".tsv"
    fname = os.path.join(csv_path, tsv_name)
    df[e].to_csv(fname, sep="\t")
    tsv_name = "EEG_data_" + e_file + ".tsv"
    fname = os.path.join(csv_path, tsv_name)
    e_events[e].to_csv(fname, sep="\t")
    tsv_name = "Semiology_data_" + e_file + ".tsv"
    fname = os.path.join(csv_path, tsv_name)
    s_events[e].to_csv(fname, sep="\t")
    tsv_name = "Testing_data_" + e_file + ".tsv"
    fname = os.path.join(csv_path, tsv_name)
    t_events[e].to_csv(fname, sep="\t")    

for idx, val in enumerate(df.keys()):
    if idx == 0:
        # all data vertical
        vconcat = df[val]
        # all data horizontal
        concat = df[val]
        source = "source_" + str(idx)
        concat[source] = val
        cols = list(concat)
        cols.insert(0, cols.pop(cols.index(source)))
        concat = concat.loc[:, cols]
        concat = concat.sort_values(by=["time_from_onset"])
        if "source" in concat.keys():
            concat.drop(columns=["source"], axis=1, inplace=True)
        concat["order_of_occurence"] = (1 + np.arange(len(concat.loc[:,"time_from_onset"])))
        # eeg, semio
        eeg_ga, semio_ga, test_ga = e_events[val], s_events[val], t_events[val]  # should be same keys as for e in edfs...

    if idx > 0:
        # all data vertical
        vnew_df = df[val]
        vconcat = pd.concat([vconcat, vnew_df], axis=0)
        # all data horizontal
        new_df = df[val]
        source = "source_" + str(idx)
        new_df[source] = val
        cols = list(new_df)
        cols.insert(0, cols.pop(cols.index(source)))
        new_df = new_df.loc[:, cols]
        if "source" in new_df.keys():
            new_df.drop(columns=["source"], axis=1, inplace=True)
        new_df["order_of_occurence"] = (1 + np.arange(len(new_df.loc[:,"time_from_onset"]))).astype(int)
        concat = pd.merge(concat, new_df, how="outer", on="description", suffixes=(" ", "  "))
        # eeg, semio
        ne, ns, nt = e_events[val], s_events[val], t_events[val]
        eeg_ga = pd.merge(eeg_ga, ne, how="outer", on="description", suffixes=(" ", "  ")) 
        semio_ga = pd.merge(semio_ga, ns, how="outer", on="description", suffixes=(" ", "  "))
        test_ga = pd.merge(test_ga, nt, how="outer", on="description", suffixes=(" ", "  "))

    idx += 1

if "source_0" in vconcat.keys():
    vconcat.drop(columns=["source_0"], axis=1, inplace=True)


# save grand averages
if win:
    eeg_ga.to_csv("..\\results\\grand_average\\tables\\EEG_data_grand_average.tsv", sep="\t")
    semio_ga.to_csv("..\\results\\grand_average\\tables\\Semiology_data_grand_average.tsv", sep="\t")
    test_ga.to_csv("..\\results\\grand_average\\tables\\Testing_data_grand_average.tsv", sep="\t")
    concat.to_csv("..\\results\\grand_average\\tables\\All_data_grand_average_horizontal.tsv", sep="\t")
    vconcat.to_csv("..\\results\\grand_average\\tables\\All_data_grand_average.tsv", sep="\t")

else:
    eeg_ga.to_csv("../results/grand_average/tables/EEG_data_grand_average.tsv", sep="\t")
    semio_ga.to_csv("../results/grand_average/tables/Semiology_data_grand_average.tsv", sep="\t")
    test_ga.to_csv("../results/grand_average/tables/Testing_data_grand_average.tsv", sep="\t")
    concat.to_csv("../results/grand_average/tables/All_data_grand_average_horizontal.tsv", sep="\t")
    vconcat.to_csv("../results/grand_average/tables/All_data_grand_average.tsv", sep="\t")

# write excel file
write_excel_table(e_events, s_events, win=win) 

----
# Interactive Visualization
----

In [ ]:
# Plots/report for single seizures
report_title = subj_name + " - Single seizure plots"
report = Report(subject=subj_name, title=report_title)

event_folders = glob.glob("../results/*")
if win:
    event_folders = glob.glob("..\\results\\*")
data = dict()
EEG = dict()
Semio = dict()
Test = dict()
interactive_plots = dict()

for e in event_folders:
    if win:
        source = e.split("\\")[-1].split(".")[0]
        sep = "\\"
    else:
        source = e.split("/")[-1].split(".")[0]
        sep = "/"
    tsv_path = join(e, "tables")
    
    tsv_name = "All_data_" + source + ".tsv"
    tsv = os.path.join(tsv_path, tsv_name)
    data[source] = pd.read_csv(tsv, sep="\t")
    tsv_name = "EEG_data_" + source + ".tsv"
    tsv = os.path.join(tsv_path, tsv_name)
    EEG[source] = pd.read_csv(tsv, sep="\t")    
    tsv_name = "Semiology_data_" + source + ".tsv"
    tsv = os.path.join(tsv_path, tsv_name)
    Semio[source] = pd.read_csv(tsv, sep="\t")
    tsv_name = "Testing_data_" + source + ".tsv"
    tsv = os.path.join(tsv_path, tsv_name)
    Test[source] = pd.read_csv(tsv, sep="\t")

    if source == "grand_average":
        pass
    else:
        interactive_plots[source] = plot_interactive_subplot_with_table(data[source], EEG[source], 
                                                                    Semio[source], Test[source], title=source)
        save_name = join("..", "results", source, "viz", str(source + "_interactive_viz.html"))
        if not os.path.isfile(save_name):
            save_plotly_to_html(interactive_plots[source], source=source)
            cap = source + " VIZ --> seizure"
            report.add_htmls_to_section(interactive_plots[source].to_html(full_html=False), 
                                        section=source, captions=cap)
        

        # event counts (plot.ly)
        event_counts = plot_interactive_eeg_and_semio(eeg=EEG[source], semio=Semio[source], source=source)
        cap = source + " VIZ --> event_conuts"
        sec = source
        report.add_htmls_to_section(event_counts.to_html(full_html=False), section=sec, captions=cap)

        # Testing
        cap = source + " VIZ --> Testing results"
        testing_viz = plot_interactive_testing_results(t_events=Test[source], title=cap)
        report.add_htmls_to_section(testing_viz.to_html(full_html=False), section=sec, captions=cap)

# Save all
report_save_name = "../results/Single_seizures_report.html"
if win:
    report_save_name = "..\\results\\Single_seizures_report.html"
report.save(report_save_name, overwrite=True)

In [ ]:
# Plots/report for grand average

ga_report_title = subj_name + " - All seizures"
ga_report = Report(subject=subj_name, title=ga_report_title)

source="grand_average"

EEG["grand_average"], Semio["grand_average"], Test["grand_average"] = extract_ordered_groups(df=data["grand_average"])

ga_fig = plot_interactive_subplot_with_table(df=data["grand_average"], eeg=EEG["grand_average"], 
                                                semio=Semio["grand_average"], testing=Test["grand_average"], title=ga_report_title)

save_name = join("..", "results", "grand_average", "viz", str("grand_average_interactive_viz.html"))
if not os.path.isfile(save_name):
    save_plotly_to_html(ga_fig, source=source)
    cap = source + " VIZ --> All seizures"
    ga_report.add_htmls_to_section(ga_fig.to_html(full_html=False), 
                                section=source, captions=cap)

# event counts (plot.ly)
event_counts = plot_interactive_eeg_and_semio(eeg=EEG[source], semio=Semio[source], source=source)
cap = source + " VIZ --> All event_conuts"
sec = source
ga_report.add_htmls_to_section(event_counts.to_html(full_html=False), section=sec, captions=cap)
# EEG
cap = source + " VIZ --> All EEG results"
eeg_viz = plot_interactive_EEG_results(e_events=EEG["grand_average"], title=cap)
ga_report.add_htmls_to_section(eeg_viz.to_html(full_html=False), section=sec, captions=cap)
# Semiology
cap = source + " VIZ --> All Testing results"
testing_viz = plot_interactive_testing_results(t_events=Test[source], title=cap)
ga_report.add_htmls_to_section(testing_viz.to_html(full_html=False), section=sec, captions=cap)
# Testing
cap = source + " VIZ --> All Semiology results"
semio_viz = plot_interactive_semio_results(s_events=Semio[source], title=cap)
ga_report.add_htmls_to_section(semio_viz.to_html(full_html=False), section=sec, captions=cap)

report_save_name = "../results/Grand_average_report.html"
if win:
    report_save_name = "..\\results\\Grand_average_report.html"
ga_report.save(report_save_name, overwrite=True)

# Blueprint for marker name translation
-----------------------------------

## Configuration comes from an excel file w fields:
- mName
- mTranslation
- mSubstitution

Goals:
- mPlainText names for all Visualizations
- Lazy grand average visualization just like existing grand average, but with birds eye 


Procedure:

load xls file via pandas

extract dicts:
- EEG
- Semio
- Modifiers
- Anatomy






In [2]:
# load configuration from excel file:
mEEG = pd.read_excel("VEEG_config.xlsx", sheet_name="EEG")
mEEG = mEEG[["mName", "mTranslation", "mSubstitution"]]
mEEG.dropna(how="all", inplace=True)
mEEG = mEEG.set_index("mName")
print(mEEG)

mSemio = pd.read_excel("VEEG_config.xlsx", sheet_name="Semio")
mSemio = mSemio[["mName", "mTranslation", "mSubstitution"]]
mSemio.dropna(how="all", inplace=True)
mSemio = mSemio.set_index("mName")

mModifiers = pd.read_excel("VEEG_config.xlsx", sheet_name="Modifiers")
mModifiers = mModifiers[["mName", "mTranslation", "mSubstitution"]]
mModifiers.dropna(how="all", inplace=True)
mModifiers = mModifiers.set_index("mName")

mAnatomy = pd.read_excel("VEEG_config.xlsx", sheet_name="Anatomy")
mAnatomy = mAnatomy[["mName", "mTranslation", "mSubstitution"]]
mAnatomy.dropna(how="all", inplace=True)
mAnatomy = mAnatomy.set_index("mName")





                                            mTranslation  mSubstitution
mName                                                                  
spike                                             Spikes            NaN
sw                                  Spike and slow waves            NaN
poly                                          Polyspikes            NaN
polysw                         Polyspikes and slow waves            NaN
shw                                          Sharp waves            NaN
ssw                                 Sharp and slow waves            NaN
theta                                              Theta            NaN
alpha                                              Alpha            NaN
beta                                                Beta            NaN
gamma                                              Gamma            NaN
delta                                              Delta            NaN
lsp                             Late significant pattern        

In [22]:
def marker_to_text(string=None, substitute=False):

    """
    Splits the input string as needed
    
    returns:
      a string in human readable format
      type: EEG, Semio, Testing
      markers_code: e-"IAmTheBaseName"
    """
  
    d = dict()
    readbable = str()
    # ignore the i- markers - not need to translate those
    if string.startswith("i-"):
        return ""
    # the rest belongs to one of three groups
    elif string.startswith("e-"):
        d["type"] = "EEG"
    elif string.startswith("s-"):
        d["type"] = "Semiology"
    else:
        d["type"] = "Testing"
  
    # this returns a list of markers and modifiers
    rex = re.findall(r"[-|+]\w*", string)
    
    # First job is to define the base 
    try:
        # base comes first
        r = rex[0].strip("-")
        rr = rex[0]
        if r in mEEG.index:
            base = mEEG.loc[str(r)][0]
        else:
            base = str(r)
        # now we can drop it from the list
        rex.remove(rr)
    except Exception as e:
        print(f"Could not determine base: {e}")
  
  
    # 2nd job: substitutions
    if substitute == True:
        for r in rex:
            r = r.split("-")[-1].split("+")[-1] 
            if r in mEEG.index:
                if mEEG.loc[str(r)][1] != None:
                    newitems = list()
                    try:
                        print(f"mEEG.loc[str(r)][1] --> {mEEG.loc[str(r)][1]}")
                        # split the substitution
                        subst = str(mEEG.loc[str(r)][1]).split("-")
                        for s in subst:
                            if not s in rex:
                                newitems.append(s)
                        for n in newitems:
                            if n != "":
                                rex.append(str("-" + n))
                        # delete r, as it has just been substituted
                        rex.remove(str("-" + r))
                    except Exception as e:
                        print(e)
            if r in mSemio.index:
                pass
            if r in mModifiers.index:
                pass
            if r in mAnatomy.index:
                pass
    print(f"rex after substitution   -->   {rex}")      
    #print(f"rex without base: {rex}")
 #   define placeholder lists
    strEEG = []
    strSemio = []
    strAna = []
    strMod = []
    strNotRecognized = []
  
    # now we can go throug the modifiers etc.
    for r in rex:
        r = r.split("-")[-1] 
        r = r.split("+")[-1]
        r = r.strip("-")     
        if r in mEEG.index:
            strEEG.append(mEEG.loc[str(r)][0])
        elif r in mSemio.index:
            strSemio.append(mSemio.loc[str(r)][0])
        elif str("+" + r) in mModifiers.index:
            strMod.append(str(mModifiers.loc[str("+" + r)][0]))
        elif str(r) in mModifiers.index:
            strMod.append(str("with " + mModifiers.loc[str(r)][0]))
        elif r in mAnatomy.index:
            strAna.append(mAnatomy.loc[str(r)][0])
        else:
            strNotRecognized.append(r)

    # make sure output order is always the same + return 
    readable = ""
    if strEEG is not []:
        for e in sorted(strEEG):
            readable += str(" " + e)
    if strSemio is not []:
        for m in sorted(strSemio):
            readable += str(" " + m)
    if strMod is not []:
        for m in sorted(strMod):
            readable += str(" " + m)
    if strAna is not []:
        for a in sorted(strAna):
            readable += str(" " + a)     
    if strNotRecognized is not []:
        for m in sorted(strNotRecognized):
            readable += str(" " + m)
      
    readable = base + " " + readable
    return readable

testTags = ["e-BIRD-r-temp-ffluct",
            "e-ASD-FZ",
            "e-sw-FZ",
            "e-maf-l-par",
            "e-LPD+F-F7",
            "e-RDA-FZ-CZ",
            "e-oirda-l-temp-F7"]  #!!! --> left left temporal temporal

for t in testTags:
    trans = marker_to_text(t, substitute=True)
    print(f"--> {trans}\n")

rex after substitution   -->   ['-r', '-temp', '-ffluct']
--> Brief potentially ictal rhythmic discharges  with Fluctuating frequency right temporal

mEEG.loc[str(r)][1] --> -b-front-cent
rex after substitution   -->   ['-b', '-front', '-cent']
--> ASD  bilateral central frontal

mEEG.loc[str(r)][1] --> -b-front-cent
rex after substitution   -->   ['-b', '-front', '-cent']
--> Spike and slow waves  bilateral central frontal

rex after substitution   -->   ['-l', '-par']
--> muscle artifact  left parietal

mEEG.loc[str(r)][1] --> -l-temp
rex after substitution   -->   ['+F', '-l', '-temp']
--> Lateralized periodic discharges  plus fast activity left temporal

mEEG.loc[str(r)][1] --> -b-front-cent
rex after substitution   -->   ['-CZ', '-b', '-front', '-cent']
--> Rhythmic delta activity  CZ bilateral central frontal

mEEG.loc[str(r)][1] --> -l-temp
rex after substitution   -->   ['-l', '-temp', '-l', '-temp']
--> Occipital intermittent rhythmic delta activity  left left temporal tempora

## To do:

EKG?

Create a radar chart of EEG and Semiology signs (r/l hemisphere: front, temp, parietal, occipital) - https://plotly.com/python/radar-chart/

